In [ ]:
import os
import os.path
import glob
import cv2
from matplotlib import pyplot as plt

In [ ]:
dird = ''
os.chdir(dird)

In [ ]:
dir_trays = ''
dirlist = [ item for item in os.listdir(dir_trays) if os.path.isdir(os.path.join(dir_trays, item)) ]
print("Number of directory: " + str(len(dirlist)))
print(dirlist)

In [ ]:
pot_names = ['A1', 'A2', 'A3', 'A4', 'A5', 
             'B1', 'B2', 'B3', 'B4', 'B5', 
             'C1', 'C2', 'C3', 'C4', 'C5', 
             'D1', 'D2', 'D3', 'D4', 'D5']

accession_names = [['Kz-9-09', 'Hs-0-09', 'Ren-11-09', 'Ws-2-09', 'Br-0-09', 'C24-09', 'CIBC-2-09', 
                        'Jm-0-09', 'Hovdala-2-09', 'Cvi-0-09', 'Ede-1-09', 'MIB-28-09', 'PHW-2-09', 'Or-0-09', 
                        'Ors-2-09', 'Hsm-09', 'Col-0-09', 'Is-1-09', 'Edr-1-09', 'Ba4-1-09']] 
                       

plant_id = [['1000393','1000394','1000395','1000396','1000397','1000398','1000399',
                '10000400','1000401','1000402','1000403','1000404','1000405','1000406','1000407',
                '1000408','1000409','1000410','1000411', '1000412']]

In [ ]:
# Base directory containing the trays
dir_trays = '' 

# List all directories in the base directory
dir_list = [item for item in os.listdir(dir_trays) if os.path.isdir(os.path.join(dir_trays, item))]
print("Number of directories:", len(dir_list))
print(dir_list)

# Select a specific directory by index
z = 3
print("Entering directory:", dir_list[z])
dir_name = os.path.join(dir_trays, dir_list[z])
tray = dir_list[z]
print("Directory path:", dir_name)

# Change current working directory to the selected directory
os.chdir(dir_name)
print("Current working directory:", os.getcwd())

# List all PNG files in the directory
file_list = [os.path.basename(item) for item in glob.glob(os.path.join(dir_name, '*.png'), recursive=True)]

# Create output directory for the final dataset if it doesn't exist
output_dir = os.path.join("../../final_dataset", tray)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate through each file in the list
for i, file_name in enumerate(file_list):
    print(i, file_name)
    filename_no_ext = os.path.splitext(file_name)[0]
    label_filename = os.path.join(dir_name, f"{filename_no_ext}.txt")
    
    # Read the label file
    with open(label_filename) as f:
        lines = f.readlines()
    
    # Read and convert the image to RGB
    img_bgr = cv2.imread(file_name)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    dh, dw, _ = img_rgb.shape
    print(file_name, "Dimensions (h, w):", dh, dw)

    # Iterate through each bounding box in the label file
    for j in range(20):
        box = lines[j].strip().split()
        class_id, x_center, y_center, w, h = map(float, box)
        print(pot_names[j], x_center, y_center, w, h)

        # Calculate bounding box coordinates
        x_center = int(x_center * dw)
        y_center = int(y_center * dh)
        w = int(w * dw)
        h = int(h * dh)
        x = x_center - w // 2
        y = y_center - h // 2
        print("Bounding box:", x_center, y_center, w, h)
        
        # Apply specific cropping adjustments based on pot name
        if pot_names[j] == 'A1':
            img_crop = img_rgb[y-4:y+h+12, x-4:x+w+5]
        elif pot_names[j] == 'B1':
            img_crop = img_rgb[y+10:y+h+5, x-4:x+w+5]
        elif pot_names[j] == 'B5':
            img_crop = img_rgb[y-12:y+h+5, x+55:x+w+35]
        elif pot_names[j] == 'D3':
            img_crop = img_rgb[y+21:y+h+15, x-4:x+w+5]
        else:
            img_crop = img_rgb[y-12:y+h+5, x-4:x+w+5]

        # Create directory for the specific pot if it doesn't exist
        pot_dir = os.path.join('../../final_dataset', tray, f'pot_{pot_names[j]}')
        if not os.path.exists(pot_dir):
            os.makedirs(pot_dir)

        # Save the cropped image
        output_filename = os.path.join(pot_dir, f"{plant_id[0][j]}_{filename_no_ext}_pot_{pot_names[j]}_{accession_names[0][j]}.png")
        cv2.imwrite(output_filename, cv2.cvtColor(img_crop, cv2.COLOR_RGB2BGR))
        print(f"Saved cropped image to {output_filename}")
